# Assignment 1

You only need to write one line of code for each question. When answering questions that ask you to identify or interpret something, the length of your response doesn’t matter. For example, if the answer is just ‘yes,’ ‘no,’ or a number, you can just give that answer without adding anything else.

We will go through comparable code and concepts in the live learning session. If you run into trouble, start by using the help `help()` function in Python, to get information about the datasets and function in question. The internet is also a great resource when coding (though note that **no outside searches are required by the assignment!**). If you do incorporate code from the internet, please cite the source within your code (providing a URL is sufficient).

Please bring questions that you cannot work out on your own to office hours, work periods or share with your peers on Slack. We will work with you through the issue.

### Classification using KNN

Let's set up our workspace and use the **Wine dataset** from `scikit-learn`. This dataset contains 178 wine samples with 13 chemical features, used to classify wines into different classes based on their origin.

The **response variable** is `class`, which indicates the type of wine. We'll use all of the chemical features to predict this response variable.

In [104]:
# Import standard libraries
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score, precision_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [105]:
from sklearn.datasets import load_wine

# Load the Wine dataset
wine_data = load_wine()

# Convert to DataFrame
wine_df = pd.DataFrame(wine_data.data, columns=wine_data.feature_names)

# Bind the 'class' (wine target) to the DataFrame
wine_df['class'] = wine_data.target

# Display Top 5 rows of DataFrame
wine_df.head()


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0


#### **Question 1:** 
#### Data inspection

Before fitting any model, it is essential to understand our data. **Use Python code** to answer the following questions about the **Wine dataset**:

_(i)_ How many observations (rows) does the dataset contain?

In [106]:
# Get the shape(rows,columns/features) of dataset
print(wine_df.shape)

(178, 14)


In [107]:
# Your answer here
print(f'Number of rows/records in data wine set is {wine_df.shape[0]}')

Number of rows/records in data wine set is 178


_(ii)_ How many variables (columns) does the dataset contain?

In [108]:
# Your answer here
print(f'Number of columns/features in data wine set is {wine_df.shape[1]}')

Number of columns/features in data wine set is 14


_(iii)_ What is the 'variable type' of the response variable `class` (e.g., 'integer', 'category', etc.)? What are the 'levels' (unique values) of the variable?

In [109]:
# Your answer here
# class/target featuer is categorical
print(f'Type of target/class feature is {wine_df["class"].unique()}')

Type of target/class feature is [0 1 2]


In [110]:
print(f'Unique class variables are {wine_df["class"].unique()}')

Unique class variables are [0 1 2]



_(iv)_ How many predictor variables do we have (Hint: all variables other than `class`)? 

In [111]:
# Your answer here
# Total number of columns is 14, so take out target variable
print(f'Number of predictor features is {len(wine_df.columns)-1}')

Number of predictor features is 13


In [112]:
# Alternate way,
# slice the dataframe except last column/target varaible and get the size of it
print(f'Number of predictor features is {len(wine_df.columns[:len(wine_df.columns)])-1}')

Number of predictor features is 13


You can use `print()` and `describe()` to help answer these questions.

#### **Question 2:** 
#### Standardization and data-splitting

Next, we must preform 'pre-processing' or 'data munging', to prepare our data for classification/prediction. For KNN, there are three essential steps. A first essential step is to 'standardize' the predictor variables. We can achieve this using the scaler method, provided as follows:

In [113]:
# Select predictors (excluding the last column)
predictors = wine_df.iloc[:, :-1]

# Standardize the predictors
scaler = StandardScaler()
predictors_standardized = pd.DataFrame(scaler.fit_transform(predictors), columns=predictors.columns)

# Display the head of the standardized predictors
print(predictors_standardized.head())

    alcohol  malic_acid       ash  alcalinity_of_ash  magnesium  \
0  1.518613   -0.562250  0.232053          -1.169593   1.913905   
1  0.246290   -0.499413 -0.827996          -2.490847   0.018145   
2  0.196879    0.021231  1.109334          -0.268738   0.088358   
3  1.691550   -0.346811  0.487926          -0.809251   0.930918   
4  0.295700    0.227694  1.840403           0.451946   1.281985   

   total_phenols  flavanoids  nonflavanoid_phenols  proanthocyanins  \
0       0.808997    1.034819             -0.659563         1.224884   
1       0.568648    0.733629             -0.820719        -0.544721   
2       0.808997    1.215533             -0.498407         2.135968   
3       2.491446    1.466525             -0.981875         1.032155   
4       0.808997    0.663351              0.226796         0.401404   

   color_intensity       hue  od280/od315_of_diluted_wines   proline  
0         0.251717  0.362177                      1.847920  1.013009  
1        -0.293321  0.406051

(i) Why is it important to standardize the predictor variables?

### without standardization, feature such as magensiusm which is in the order of 100's has higher bearing/influence on the outcome/target compared to feature such as ash which is in the order of 0.2 or something like that. To even out all the features on same scale between 0 to 1 or -1 to 1, we standardize all the numerical features/variables except target varaibles. If there any categorical features, they need to be encoded using one-hot encoding preferable or dummy varaibles(not preferred)

(ii) Why did we elect not to standard our response variable `Class`?

## since we don't use target/response variable in training the model, so it's not required to standardize the response variable

(iii) A second essential step is to set a random seed. Do so below (Hint: use the random.seed function). Why is setting a seed important? Is the particular seed value important? Why or why not?

### setting random seed value guarantees that running python notebook code will gurarnate to give the same predictable result. it's like throwing biased dice or coin which will always gives same result regardless whoever throws it

(iv) A third essential step is to split our standardized data into separate training and testing sets. We will split into 75% training and 25% testing. The provided code randomly partitions our data, and creates linked training sets for the predictors and response variables. 

Extend the code to create a non-overlapping test set for the predictors and response variables.

In [114]:
# set a seed for reproducibility
np.random.seed(42)

# split the data into a training and testing set. hint: use train_test_split !
wine_train,wine_test = train_test_split(wine_df,test_size=0.25,shuffle=True,stratify=wine_df['class'])
# Your code here ...
X_train = wine_train[:len(wine_train)]
y_train = wine_train['class']
X_test = wine_test[:len(wine_test)]
y_test = wine_test['class']


In [115]:
# checking the top 5 rows of train data
wine_train.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
8,14.83,1.64,2.17,14.0,97.0,2.8,2.98,0.29,1.98,5.20,1.08,2.85,1045.0,0
104,12.51,1.73,1.98,20.5,85.0,2.2,1.92,0.32,1.48,2.94,1.04,3.57,672.0,1
36,13.28,1.64,2.84,15.5,110.0,2.6,2.68,0.34,1.36,4.60,1.09,2.78,880.0,0
78,12.33,0.99,1.95,14.8,136.0,1.9,1.85,0.35,2.76,3.40,1.06,2.31,750.0,1
2,13.16,2.36,2.67,18.6,101.0,2.8,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0


In [116]:
# Checking the top 5 rows of Test data
wine_test.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
35,13.48,1.81,2.41,20.5,100.0,2.70,2.98,0.26,1.86,5.10,1.04,3.47,920.0,0
93,12.29,2.83,2.22,18.0,88.0,2.45,2.25,0.25,1.99,2.15,1.15,3.30,290.0,1
7,14.06,2.15,2.61,17.6,121.0,2.60,2.51,0.31,1.25,5.05,1.06,3.58,1295.0,0
28,13.87,1.90,2.80,19.4,107.0,2.95,2.97,0.37,1.76,4.50,1.25,3.40,915.0,0
87,11.65,1.67,2.62,26.0,88.0,1.92,1.61,0.40,1.34,2.60,1.36,3.21,562.0,1


In [117]:
print('Feature of training data',X_train.head())

Feature of training data      alcohol  malic_acid   ash  alcalinity_of_ash  magnesium  total_phenols  \
8      14.83        1.64  2.17               14.0       97.0            2.8   
104    12.51        1.73  1.98               20.5       85.0            2.2   
36     13.28        1.64  2.84               15.5      110.0            2.6   
78     12.33        0.99  1.95               14.8      136.0            1.9   
2      13.16        2.36  2.67               18.6      101.0            2.8   

     flavanoids  nonflavanoid_phenols  proanthocyanins  color_intensity   hue  \
8          2.98                  0.29             1.98             5.20  1.08   
104        1.92                  0.32             1.48             2.94  1.04   
36         2.68                  0.34             1.36             4.60  1.09   
78         1.85                  0.35             2.76             3.40  1.06   
2          3.24                  0.30             2.81             5.68  1.03   

     od280/od

In [118]:
print('Target/class variable of traing',y_train.head())

Target/class variable of traing 8      0
104    1
36     0
78     1
2      0
Name: class, dtype: int32


In [119]:
print(f'predictor variables of Test data {X_test.head()}')

predictor variables of Test data     alcohol  malic_acid   ash  alcalinity_of_ash  magnesium  total_phenols  \
35    13.48        1.81  2.41               20.5      100.0           2.70   
93    12.29        2.83  2.22               18.0       88.0           2.45   
7     14.06        2.15  2.61               17.6      121.0           2.60   
28    13.87        1.90  2.80               19.4      107.0           2.95   
87    11.65        1.67  2.62               26.0       88.0           1.92   

    flavanoids  nonflavanoid_phenols  proanthocyanins  color_intensity   hue  \
35        2.98                  0.26             1.86             5.10  1.04   
93        2.25                  0.25             1.99             2.15  1.15   
7         2.51                  0.31             1.25             5.05  1.06   
28        2.97                  0.37             1.76             4.50  1.25   
87        1.61                  0.40             1.34             2.60  1.36   

    od280/od315_o

In [120]:
print(f'Response variable of Test data {y_test.head()}')

Response variable of Test data 35    0
93    1
7     0
28    0
87    1
Name: class, dtype: int32


#### **Question 3:**
#### Model initialization and cross-validation
We are finally set to fit the KNN model. 


Perform a grid search to tune the `n_neighbors` hyperparameter using 10-fold cross-validation. Follow these steps:

1. Initialize the KNN classifier using `KNeighborsClassifier()`.
2. Define a parameter grid for `n_neighbors` ranging from 1 to 50.
3. Implement a grid search using `GridSearchCV` with 10-fold cross-validation to find the optimal number of neighbors.
4. After fitting the model on the training data, identify and return the best value for `n_neighbors` based on the grid search results.

In [126]:
grid_parameter = {
    'n_neighbors':range(1,50)
}

In [127]:
model = KNeighborsClassifier(n_neighbors=5)

In [129]:
classifier = GridSearchCV(estimator=model,param_grid=grid_parameter,cv=5)

In [130]:
# Your code here...
classifier.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(1, 50)})

In [134]:
# Best Neight neighbour number is below
print("Best Nearest Neighbour number is ",classifier.best_params_)

Best Nearest Neighbour number is  {'n_neighbors': 25}


#### **Question 4:**
#### Model evaluation

Using the best value for `n_neighbors`, fit a KNN model on the training data and evaluate its performance on the test set using `accuracy_score`.

In [135]:
# Your code here...
model = KNeighborsClassifier(n_neighbors=25)

In [136]:
model.fit(X_train,y_train)

KNeighborsClassifier(n_neighbors=25)

In [ ]:
accuracy_score(y_true=y_test)

# Criteria


| **Criteria**                                           | **Complete**                                      | **Incomplete**                                    |
|--------------------------------------------------------|---------------------------------------------------|--------------------------------------------------|
| **Data Inspection**                                    | Data is inspected for number of variables, observations and data types. | Data inspection is missing or incomplete.         |
| **Data Scaling**                                       | Data scaling or normalization is applied where necessary (e.g., using `StandardScaler`). | Data scaling or normalization is missing or incorrectly applied. |
| **Model Initialization**                               | The KNN model is correctly initialized and a random seed is set for reproducibility.            | The KNN model is not initialized, is incorrect, or lacks a random seed for reproducibility. |
| **Parameter Grid for `n_neighbors`**                   | The parameter grid for `n_neighbors` is correctly defined. | The parameter grid is missing or incorrectly defined. |
| **Cross-Validation Setup**                             | Cross-validation is set up correctly with 10 folds. | Cross-validation is missing or incorrectly set up. |
| **Best Hyperparameter (`n_neighbors`) Selection**       | The best value for `n_neighbors` is identified using the grid search results. | The best `n_neighbors` is not selected or incorrect. |
| **Model Evaluation on Test Data**                      | The model is evaluated on the test data using accuracy. | The model evaluation is missing or uses the wrong metric. |


## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Note:

If you like, you may collaborate with others in the cohort. If you choose to do so, please indicate with whom you have worked with in your pull request by tagging their GitHub username. Separate submissions are required.

### Submission Parameters:
* Submission Due Date: `11:59 PM - 05/18/2025`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_3.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/LCR/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-6-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.
